In [1]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
# add the path of gotmtool
sys.path.append("../gotmtool")
from gotmtool import *
from gotmtool.diags import *

In [2]:
# set gotm
# gotm_root = '/Volumes/WD02/SaildroneFlux/gotm/run/ERA5_plus_STD-2h'
gotm_root = os.environ['HOME']+'/scratch/saildrone_southern_ocean/gotm/run/ERA5_plus_STD-2h'
prefix_list = [filename for filename in os.listdir(gotm_root) if filename.startswith('C')]
filename = 'gotm_out.nc'

In [3]:
tidx = -24 # last day for hourly output
dat = {}
clabels = set()
mlabels = set()
hlabels = set()
tlabels = set()
ncases = len(prefix_list)
for i, casename in enumerate(prefix_list):
    if i%int(ncases/20) == 0:
        print('{:3d} %'.format(round(i/ncases*100)))
#     print(casename)
    casestring, mlabel = casename.split('_')
    clabel, _, hlabel, _, tlabel = casestring.split('-')
    sim = Simulation(path=gotm_root+'/'+casename)
    dataset = sim.load_data()
    heatflux = dataset.data_vars['heat'][0,0,0].values
    tau = dataset.data_vars['tx'][0,0,0].values
    rho = dataset.data_vars['rho'][:,:,0,0]
    time = dataset.time
    mld = -get_mld_deltaR(rho)
    mldavg = mld[tidx:].mean().values
    tmp = {
        'clabel': clabel,
        'hlabel': hlabel,
        'tlabel': tlabel,
        'mlabel': mlabel,
        'heat':   float(heatflux),
        'tau':    float(tau),
        'mldavg': float(mldavg),
        }
    dat[casename] = tmp
    clabels.add(clabel)
    mlabels.add(mlabel)
    hlabels.add(hlabel)
    tlabels.add(tlabel)

  0 %
  5 %
 10 %
 15 %
 20 %
 25 %
 30 %
 35 %
 40 %
 45 %
 50 %
 55 %
 60 %
 65 %
 70 %
 75 %
 80 %
 85 %
 90 %
 95 %
100 %


In [7]:
import pickle
pkl_file = 'mld.pkl'
with open(pkl_file, 'wb') as pklfile:
    pickle.dump(dat, pklfile)

In [8]:
import csv
csv_columns = ['clabel', 'hlabel', 'tlabel', 'mlabel', 'heat', 'tau', 'mldavg']
csv_file = 'mld.csv'

with open(csv_file, 'w') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
    writer.writeheader()
    for key in dat.keys():
        writer.writerow(dat[key])